In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from scipy.stats import pointbiserialr
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import pearsonr
import statsmodels.api as sm
import numpy as np
from scipy.stats import shapiro, levene, f_oneway , chi2 , kruskal , t , mannwhitneyu
from statsmodels.stats.anova import AnovaRM
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from scipy.stats import f
from scipy.stats import ttest_ind
from scipy.stats import kstest
import pingouin as pg
from statsmodels.stats.multitest import multipletests
import scikit_posthocs as sp

In [19]:
df = pd.read_csv('data.csv')
df.head()

,id,group,quest_timing,fps_wk_hr,total_wk_hr,roleplay_wk_hr,strategy_wk_hr,years_play,gaming_ability_connection,media_reports,...,digit_1_max,digit_2_max,average_digit_max,group_by_wk_hr,average_digit_correct,filter_$,group_by_fps_wk_hr,choice_mean_correct,simon_mean_correct,alt_mean_correct
0,1,1,1,2,3,3,3,5,7,2,...,5.0,9.0,7.0,#NULL!,5.5,1,1.00,24.0,20.00,21.00
1,2,1,1,3,4,1,1,5,10,2,...,4.0,4.0,4.0,#NULL!,4.0,1,#NULL!,23.5,22.50,23.50
2,3,1,1,1,4,1,3,5,9,1,...,6.0,6.0,6.0,#NULL!,5.5,1,1.00,24.0,22.50,22.00
3,4,1,1,3,4,1,2,5,9,1,...,6.0,6.0,6.0,#NULL!,4.0,1,#NULL!,23.5,22.00,22.00
4,5,1,1,6,4,2,2,5,8,1,...,6.0,8.0,7.0,#NULL!,5.0,1,2.00,23.5,21.50,20.50


In [20]:
from scipy.stats import f_oneway, mannwhitneyu

choice_rt_data = df[['group', 'quest_timing', 'choice_mean_RT', 'choice_total_correct','choice_mean_correct']]

# Perform two-way ANOVA for mean RT
rt_anova = choice_rt_data.groupby(['group', 'quest_timing']).mean().reset_index()
print("One way anova for Questionnaire Timing Effect on Choice RT")
anova_result = f_oneway(rt_anova[rt_anova['quest_timing'] == 1]['choice_mean_RT'],
                        rt_anova[rt_anova['quest_timing'] == 2]['choice_mean_RT'])

# Calculate mean RT
mean_rt = choice_rt_data['choice_mean_RT'].mean()
mean_rt_std = choice_rt_data['choice_mean_RT'].std()

# Perform Mann-Whitney U test for mean correct responses
before_gamers = choice_rt_data[(choice_rt_data['quest_timing'] == 1) & (choice_rt_data['group'] == 1)]['choice_mean_correct']
after_gamers = choice_rt_data[(choice_rt_data['quest_timing'] == 2) & (choice_rt_data['group'] == 1)]['choice_mean_correct']
gamers = choice_rt_data[(choice_rt_data['group'] == 1)]['choice_mean_correct']
before_non_gamers = choice_rt_data[(choice_rt_data['quest_timing'] == 1) & (choice_rt_data['group'] == 2)]['choice_mean_correct']
after_non_gamers = choice_rt_data[(choice_rt_data['quest_timing'] == 2) & (choice_rt_data['group'] == 2)]['choice_mean_correct']
non_gamers = choice_rt_data[(choice_rt_data['group'] == 2)]['choice_mean_correct']

mwu_before = mannwhitneyu(before_gamers, after_gamers)
mwu_after = mannwhitneyu(before_non_gamers, after_non_gamers)
mwu_total = mannwhitneyu(gamers, non_gamers)

# Calculate Np^2 for ANOVA
N = len(choice_rt_data)
k = len(choice_rt_data['quest_timing'].unique()) * len(choice_rt_data['group'].unique())
Np_squared = (k - 1) / (N - k)

print("Choice-RT task")
print("\nMean RT")
print(
      f"F(1, 127) = {anova_result.statistic:.2f}, p = {anova_result.pvalue:.2f}, ηp^2 = {Np_squared:.2f} ",
      f"The mean choice RT was {mean_rt:.2f} ± {mean_rt_std:.2f} ms.")

print("\nMean correct responses")
print(
      f"(Mann-Whitney U = {mwu_before.statistic:.2f}, p = {mwu_before.pvalue:.2f}; "
      f"mean: {gamers.mean():.2f} ± {gamers.std():.2f}) "
      f"and non-gamers (Mann-Whitney U = {mwu_after.statistic:.2f}, p = {mwu_after.pvalue:.2f}; "
      f"mean: {non_gamers.mean():.2f} ± {non_gamers.std():.2f}). "
      f"(Mann-Whitney U = {mwu_total.statistic:.2f}, p = {mwu_total.pvalue:.2f}).")

model = ols('choice_mean_RT ~ C(group) + C(quest_timing) + C(group):C(quest_timing)', data=choice_rt_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print("Two-way ANOVA for Group and Questionnaire Timing Effects:")
print(anova_table)


One way anova for Questionnaire Timing Effect on Choice RT
Choice-RT task

Mean RT
F(1, 127) = 0.84, p = 0.46, ηp^2 = 0.02  The mean choice RT was 368.27 ± 57.78 ms.

Mean correct responses
(Mann-Whitney U = 794.00, p = 0.87; mean: 23.78 ± 0.37) and non-gamers (Mann-Whitney U = 1499.50, p = 0.77; mean: 23.71 ± 0.48). (Mann-Whitney U = 4618.00, p = 0.25).
Two-way ANOVA for Group and Questionnaire Timing Effects:
                                 sum_sq     df         F    PR(>F)
C(group)                    3345.170352    1.0  0.998631  0.318962
C(quest_timing)             1853.141633    1.0  0.553217  0.457960
C(group):C(quest_timing)    2770.375229    1.0  0.827038  0.364327
Residual                  613005.398037  183.0       NaN       NaN
